# ROSBAG reader helper notebook for ROSBAG 2 files

The notebook helps to read rosbag 2 files, and collect mesages for the same timestamp. Please install dependencies first!

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
from rosbags.rosbag2 import Reader
from rosbags.serde import deserialize_cdr
import itertools
import collections
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
filename = '/tmp/rosbag/'
with Reader(filename) as reader:
    print(reader.start_time/1e9)
    print(reader.end_time/1e9)

In [ ]:
data_all = collections.OrderedDict()
with Reader(filename) as reader:
    for connection, timestamp, rawdata in itertools.islice(reader.messages(start = 1712752798_910_351_800),50):  # start = 1712752801_000_000_000
        try:
            b = deserialize_cdr(rawdata, connection.msgtype)
            ts = b.header.stamp.sec * 1000000000 + b.header.stamp.nanosec

            if ts not in data_all:
                data_all[ts] = {}

            data_all[ts][connection.topic] = b
        except:
            pass

data = {k: v for k,v in data_all.items() if '/camera/aligned_depth_to_color/image_raw' in v and 
                                        '/camera/color/image_raw' in v and
                                         '/camera/aligned_depth_to_color/camera_info' in v }
                
for k,v in data.items():
    print(k)
    for o in v.keys():
        print(f'  {o}')

In [ ]:
data_entry = data[list(data.keys())[0]]
img_raw = data_entry['/camera/color/image_raw']
color_img = np.frombuffer(img_raw.data, dtype = np.uint8).reshape(img_raw.height, img_raw.width,3)
depth_raw = data_entry['/camera/aligned_depth_to_color/image_raw']
depth_img = np.frombuffer(depth_raw.data, dtype = np.uint16).reshape(depth_raw.height, depth_raw.width)
Kdepth = data_entry['/camera/aligned_depth_to_color/camera_info'].k.reshape(3,3)

In [ ]:
plt.imshow(color_img)

In [ ]:
cv2.imwrite('../asset/example_image.png',color_img)
cv2.imwrite('../asset/example_depth.png',depth_img)

In [ ]:
Kdepth